Experimental Identification of Physically Feasible Dynamic Parameters of the 7-link WAM™ Robot using LMI–SDP Techniques
=======================================================================================================================

[![DOI](https://zenodo.org/badge/920/cdsousa/wam7_dyn_ident.png)](http://dx.doi.org/10.5281/zenodo.10537)

*Experimental code and data associated with the paper:*

**Cristóvão D. Sousa and Rui Cortesão, "Physical feasibility of robot base inertial parameter identification: A linear matrix inequality approach," The International Journal of Robotics Research, vol. 33, no. 6, pp. 931–944, May. 2014, doi: [10.1177/0278364913514870](http://dx.doi.org/10.1177/0278364913514870)**

[Postprint availabe from ResearchGate](http://www.researchgate.net/publication/262483999_Physical_feasibility_of_robot_base_inertial_parameter_identification_A_linear_matrix_inequality_approach)

------------------------


Authors
-------

- Cristóvão D. Sousa, [crisjss@gmail.com](mailto:crisjss@gmail.com)
- Rui Cortesão, [cortesao@isr.uc.pt](mailto:cortesao@isr.uc.pt)

------------------------


Research Notebook
-----------------

The research is done in [Python](http://www.python.org/) within an [IPython notebook](http://ipython.org/notebook.html) (the *WAM7 Dynamic Parameter Identification.ipynb* file).
Data is in *data* folder. Additional Python support code is in *support_funcs* folder.

The file *WAM7 Dynamic Parameter Identification.html* is a **preview** of the notebook. **It can be seen online at:**

**[Experimental WAM Robot Dynamic Identification with LMI](http://goo.gl/dOPj8j)**



How to run the code
-------------------

- clone the git repository (it contains code and data) from https://github.com/cdsousa/wam7_dyn_ident
- open the *WAM7 Dynamic Parameter Identification.ipynb* notebook with [IPython](http://ipython.org/)
- edit and run the code

Dependencies:

- [Python](http://www.python.org/)
- [IPython](http://ipython.org/)
- [SymPy](http://sympy.org/)
- [Numpy](http://www.numpy.org/)
- [SciPy](http://www.scipy.org/)
- [SymPyBotics](https://github.com/cdsousa/SymPyBotics)
- [PyLMI-SDP](https://github.com/cdsousa/PyLMI-SDP)

------------------------


Questions & Feedback
--------------------

Feel free to contact the authors at [crisjss@gmail.com](mailto:crisjss@gmail.com)

------------------------


License
-------

Copyright (c) 2013, Cristóvão Duarte Sousa, Rui Cortesão

All rights reserved.

[![Creative Commons License](http://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png)](http://creativecommons.org/licenses/by-nc-sa/4.0/)
This work is licensed under a [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License](http://creativecommons.org/licenses/by-nc-sa/4.0/)


---------------

---------------

---------------

---------------

# Initializations

In [ ]:
datafolder = 'data/'
tmpfolder = 'tmp/'

In [ ]:
from sympy import init_printing
init_printing()

%pylab inline

1 - Robot Model
===============

In [ ]:
import sympybotics

In [ ]:
from pathlib import Path
rbt_saved = Path(tmpfolder + 'robotmodels/wam7_model.pkl').exists()

#### Robot Definition

In [ ]:
if not rbt_saved:
    rbtdef = sympybotics.RobotDef("WAM Arm 7 DOF",
                [("-pi/2", 0, 0, "q"),
                 ("pi/2", 0, 0, "q"),
                 ("-pi/2", 0.045, 0.55, "q"),
                 ("pi/2", -0.045, 0, "q"),
                 ("-pi/2", 0, 0.3, "q"),
                 ("pi/2", 0, 0, "q"),
                 (0, 0, 0.06, "q")],
                dh_convention="standard")

    rbtdef.frictionmodel = {'Coulomb', 'viscous', 'offset'}
    rbtdef.driveinertiamodel = 'simplified'

#### Model Generation

In [ ]:
if not rbt_saved:
    %time rbt = sympybotics.RobotDynCode(rbtdef)
    %time rbt.calc_base_parms()

In [ ]:
if not rbt_saved:
    import pickle
    Path(tmpfolder + 'robotmodels').mkdir(parents=True, exist_ok=True)

    with open(tmpfolder + 'robotmodels/wam7_model.pkl', 'wb') as file:
        pickle.dump( rbt, file )

## Base Dynamic Parameters

In [ ]:
import pickle
with open(tmpfolder +  'robotmodels/wam7_model.pkl', 'rb' ) as file:
          rbt = pickle.load( file )

In [ ]:
rbt.dyn.n_dynparms, rbt.dyn.n_base

In [ ]:
rbt.dyn.baseparms.n()

2 - Trajectories and Recorded Data
==================================

In [ ]:
import pickle
import sympybotics
import numpy

In [ ]:
with open(tmpfolder +  'robotmodels/wam7_model.pkl', 'rb' ) as file:
          rbt = pickle.load( file )

### Plots

In [ ]:
parms_traj = 'traj1'

with open(datafolder +  'trajectories/%s_shwfl_abq0.pkl'%parms_traj, 'rb' ) as file:
    s_h_wf_l, a_b_q0 = pickle.loads(file.read().replace(b'\r\n', b'\n'), encoding='latin1')

In [ ]:
h = 0.001
decimate = 10
h_plot = h*decimate

with open(datafolder +  'trajectories/%s.dat'%parms_traj, 'r' ) as file:
          q_ref_orig = numpy.loadtxt(file)
s = round(q_ref_orig.shape[0] / decimate)

q_ref = numpy.zeros((s, rbt.dof))
for i in range(s):
    q_ref[i, :] = q_ref_orig[i*decimate, :]

In [ ]:
si = round(1/h_plot)
sf = round(si + 20/h_plot)
t = numpy.arange(sf-si) * h_plot

In [ ]:
from matplotlib import pyplot as plt
plt.close()

for i in range(rbt.dof):
    plt.plot(t,q_ref[si:sf,i], label="$q_%d$"%(i+1))
plt.legend()

plt.xlabel("Time (s)")
plt.ylabel("Joint positions (rad)")

plt.show()

3 - Regression Data Processing
==============================

In [ ]:
import os
import pickle
import math
import sympy
import numpy
import sympybotics
from pathlib import Path

In [ ]:
with open(tmpfolder +  'robotmodels/wam7_model.pkl', 'rb' ) as file:
    rbt = pickle.load(file)

### Data Load

In [ ]:
parms_traj = 'traj1'

In [ ]:
from support_funcs.regr_data_proc import read_data
t_raw, q_raw, tau_raw, t_ref, q_ref = read_data(rbt.dof, 0.001,
                                                datafolder + 'recdata/rbtlog_%s.dat'%parms_traj,
                                                datafolder + 'trajectories/traj1.dat')

### Some Plots

In [ ]:
from matplotlib import pyplot as plt
plt.close()

for i in range(q_raw.shape[1]):
    plt.plot(t_raw,q_raw[:,i])

plt.show()

In [ ]:
from matplotlib import pyplot as plt
plt.close()

joint = 7
plt.plot(t_ref+0.20,q_ref[:,joint-1])
plt.plot(t_raw,q_raw[:,joint-1])

plt.show()

In [ ]:
del t_raw, q_raw, tau_raw, t_ref, q_ref

### Parameter Load

In [ ]:
with open(datafolder +  'trajectories/%s_shwfl_abq0.pkl'%parms_traj, 'rb' ) as file:
    s_h_wf_l, a_b_q0 = pickle.loads(file.read().replace(b'\r\n', b'\n'), encoding='latin1')

print(dict(zip(('S', 'h', 'wf', 'L'), s_h_wf_l)))
print('\n')

### Filter Cut-off Frequency Definition

In [ ]:
fc_mult = 10

In [ ]:
wf = float(s_h_wf_l[2])
L = float(s_h_wf_l[3])
fc = fc_mult * ( wf / (2*math.pi) * L )
fc

In [ ]:
fc_q = fc
fc_dq = fc
fc_ddq = fc

fc_tau = float('Inf')  # fc

si = int(3/0.001)
sf = -int(1/0.001)

### Data Filtering, Differentiation and Regression Data Generation

In [ ]:
rec_h = 0.001

In [ ]:
from support_funcs.regr_data_proc import read_data, diff_and_filt_data, gen_regr_matrices

def process_regression_data(traj_name):
    t_raw, q_raw, tau_raw, t_ref, q_ref = read_data(dof=rbt.dof, h=rec_h,
                                                    rbtlogfile=datafolder + 'recdata/rbtlog_%s.dat'%traj_name,
                                                    trajreffile=datafolder + 'trajectories/%s.dat'%traj_name)
    
    q, dq, ddq, tau = diff_and_filt_data(dof=rbt.dof, h=rec_h,  q_raw=q_raw, tau_raw=tau_raw,
                                   fc_q=fc_q, fc_dq=fc_dq, fc_ddq=fc_ddq, fc_tau=fc_tau)
    
    t_raw = t_raw[si:sf]; q_raw = q_raw[si:sf]; tau_raw = tau_raw[si:sf]
    q = q[si:sf]; dq = dq[si:sf]; ddq = ddq[si:sf]; tau = tau[si:sf]
    
    t_raw = t_raw - t_raw[0]
    
    t = numpy.array(range(q.shape[0])) * rec_h
    
    W, omega, Q1, R1, rho1 = gen_regr_matrices(rbt, q, dq, ddq, tau)
    
    return t_raw, q_raw, tau_raw, t, q, dq, ddq, tau, W, omega, Q1, R1, rho1

In [ ]:
def proc_data_saved(tmpfolder, traj):
    raw_saved = Path(tmpfolder + 'procdata/' + traj + '_raw.npz').exists()
    proc_saved = Path(tmpfolder + 'procdata/' + traj + '_proc.npz').exists()
    regr_saved = Path(tmpfolder + 'procdata/' + traj + '_regr.npz').exists()
    return (raw_saved and proc_saved and regr_saved)

In [ ]:
Path(tmpfolder + 'procdata').mkdir(parents=True, exist_ok=True)

In [ ]:
from support_funcs.utils import _fprint

for traj in ['traj1', 'traj2', 'traj3', 'traj4']:
    if not proc_data_saved(tmpfolder, traj):
        _fprint('>>>>>>>> %s proc <<<<<<<<<'%traj)
        %time t_raw, q_raw, tau_raw, t, q, dq, ddq, tau, W, omega, Q1, R1, rho1 = process_regression_data(traj)

        _fprint(' (cond=%f)'%numpy.linalg.cond(W))

        _fprint('%s save'%traj)

        %time \
        numpy.savez_compressed(tmpfolder + 'procdata/' + traj + '_raw', t=t_raw, q=q_raw, tau=tau_raw);\
        numpy.savez_compressed(tmpfolder + 'procdata/' + traj + '_proc', t=t, q=q, dq=dq, ddq=ddq, tau=tau); \
        numpy.savez_compressed(tmpfolder + 'procdata/' + traj + '_regr', W=W, omega=omega, Q1=Q1, R1=R1, rho1=rho1)

        del t_raw, q_raw, tau_raw, t, q, dq, ddq, tau, W, omega, Q1, R1, rho1
    else:
        _fprint('>>>>>>>> %s already saved <<<<<<<<<'%traj)

### Plot

In [ ]:
raw = numpy.load(tmpfolder + 'procdata/traj1_raw.npz')
tau_raw = raw['tau']
q_raw = raw['q']
del raw
proc = numpy.load(tmpfolder + 'procdata/traj1_proc.npz')
t = proc['t']
tau = proc['tau']
q = proc['q']
dq = proc['dq']
del proc

In [ ]:
from matplotlib import pyplot as plt
plt.close()

joint = 1
firsts = 20000
plt.plot(t[:firsts],q_raw[:,joint-1][:firsts])
#plt.plot(t[:firsts],q[:,joint-1][:firsts])
#plt.plot(t[:firsts],dq[:,joint-1][:firsts])
plt.plot(t[:firsts],tau_raw[:,joint-1][:firsts])

plt.show()

In [ ]:
del t, q, dq, tau, q_raw, tau_raw

4 - Dynamic Parameter Estimation
================================

In [ ]:
import os
import pickle
import sympy
import numpy
import time
from pathlib import Path

import sympybotics
import lmi_sdp

from lmi_sdp import init_lmi_latex_printing
init_lmi_latex_printing()

In [ ]:
with open(tmpfolder +  'robotmodels/wam7_model.pkl', 'rb' ) as file:
    rbt = pickle.load( file )

In [ ]:
epsilon_safemargin = 1e-6
epsilon_sdptol = 1e-7

## LMI Matrix Definitions

In [ ]:
dof = rbt.dof

delta = rbt.dyn.dynparms
n_delta = rbt.dyn.n_dynparms

beta = rbt.dyn.baseparms.n()
n_beta = rbt.dyn.n_base
beta_symbs = sympy.Matrix([sympy.Symbol('beta'+str(i+1),real=True) for i in range(n_beta)])

delta_d = (rbt.dyn.Pd.T*delta)
n_delta_d = len(delta_d)

Pb = rbt.dyn.Pb

varchange_dict = dict(zip(Pb.T*delta ,  beta_symbs - ( beta - Pb.T*delta )))

In [ ]:
from support_funcs.utils import skew, mrepl
from sympy import BlockMatrix, Matrix, eye, Identity
I = Identity
S = skew

In [ ]:
D_inertia_blocks = []
for i in range(dof):
    L = rbt.rbtdef.L[i]
    l = rbt.rbtdef.l[i]
    m = rbt.rbtdef.m[i]
    Di = BlockMatrix([[L,    S(l).T],
                      [S(l), I(3)*m]])
    D_inertia_blocks.append(Di.as_explicit())

In [ ]:
D_inertia_blocks[0]

In [ ]:
D_other_blocks = []
for i in range(dof):
    if rbt.rbtdef.driveinertiamodel == 'simplified':
        D_other_blocks.append( Matrix([rbt.rbtdef.Ia[i]]) )
    if 'viscous' in rbt.rbtdef.frictionmodel:
        D_other_blocks.append( Matrix([rbt.rbtdef.fv[i]]) )
    if 'Coulomb' in rbt.rbtdef.frictionmodel:
        D_other_blocks.append( Matrix([rbt.rbtdef.fc[i]]) )

In [ ]:
D_blocks = D_inertia_blocks + D_other_blocks

In [ ]:
D_blocks

## Change of Variable Space

In [ ]:
varchange_dict = dict(zip(Pb.T*delta ,  beta_symbs - ( beta - Pb.T*delta )))

In [ ]:
DB_blocks = [mrepl(Di, varchange_dict) for Di in D_blocks]

## LMI Definitions

In [ ]:
from lmi_sdp import LMI_PD, LMI

In [ ]:
DB_LMIs = list(map(LMI_PD, DB_blocks))

In [ ]:
DB_LMIs[0]

In [ ]:
DB_LMIs_marg = list(map(lambda lm: LMI(lm, epsilon_safemargin*eye(lm.shape[0])) , DB_blocks))

In [ ]:
DB_LMIs_marg[0]

In [ ]:
DB_LMIs_marg[0].canonical()

## SDP Solver Interfaces

### [SDPA](http://sdpa.sourceforge.net/)

In [ ]:
def sdpa(objf, lmis, variables):
    sdpadat = lmi_sdp.to_sdpa_sparse(objf, lmis, variables)
    with open(tmpfolder + 'sdpa_dat/sdp.dat-s', 'w') as f:
        f.write(sdpadat)
    
    stdout = !sdpa -ds "$tmpfolder"sdpa_dat/sdp.dat-s -o "$tmpfolder"sdpa_dat/sdpa.out -p "$tmpfolder"sdpa_dat/param.sdpa -pt 2
    print('\n'.join(stdout))

    try:
        outfile = open(tmpfolder+'sdpa_dat/sdpa.out', 'r').read().splitlines()
        sol = [float(v) for v in outfile[outfile.index('xVec = ') + 1].replace('{', '').replace('}', '').split(',')]
        return numpy.matrix(sol).T
    except:
        return

### [CSDP](https://github.com/coin-or/Csdp/wiki)

In [ ]:
def csdp(objf, lmis, variables):
    sdpadat = lmi_sdp.to_sdpa_sparse(objf, lmis, variables)
    with open(tmpfolder + 'sdpa_dat/sdp.dat-s', 'w') as f:
        f.write(sdpadat)
    
    stdout = !csdp "$tmpfolder"sdpa_dat/sdp.dat-s "$tmpfolder"sdpa_dat/csdp.out
    print('\n'.join(stdout))

    try:
        outfile = open(tmpfolder+'sdpa_dat/csdp.out', 'r').read().splitlines()
        sol = [float(v) for v in outfile[0].split()]
        return numpy.matrix(sol).T
    except:
        return

### [DSDP5](https://www.mcs.anl.gov/hs/software/DSDP/)
[https://doi.org/10.1145/1356052.1356057](https://doi.org/10.1145/1356052.1356057)

In [ ]:
def dsdp5(objf, lmis, variables):
    sdpadat = lmi_sdp.to_sdpa_sparse(objf, lmis, variables)
    with open(tmpfolder + 'sdpa_dat/sdp.dat-s', 'w') as f:
        f.write(sdpadat)
    
    stdout = !dsdp5 "$tmpfolder"sdpa_dat/sdp.dat-s -save "$tmpfolder"sdpa_dat/dsdp5.out -gaptol $epsilon_sdptol
    print('\n'.join(stdout))

    try:
        outfile = open(tmpfolder+'sdpa_dat/dsdp5.out', 'r').read().splitlines()
        sol = [float(v) for v in outfile[0].split()]
        return numpy.matrix(sol).T
    except:
        return

### [DSDP5 through CVXOPT](https://cvxopt.org/userguide/coneprog.html#cvxopt.solvers.sdp)

In [ ]:
def cvxopt_dsdp5(objf, lmis, variables):
    import cvxopt.solvers
    c, Gs, hs = lmi_sdp.to_cvxopt(objf, lmis, variables)
    cvxopt.solvers.options['DSDP_GapTolerance'] = epsilon_sdptol
    tic = time.time()
    sdpout = cvxopt.solvers.sdp(c, Gs=Gs, hs=hs, solver='dsdp')
    toc = time.time()
    print(sdpout['status'], ('ATT!: \'optimal\' does not necessarily mean feasible'))
    print('Elapsed time: %.2f s'%(toc-tic))
    return numpy.matrix(sdpout['x'])

### [CVXOPT CONELP](https://cvxopt.org/userguide/coneprog.html#cvxopt.solvers.sdp)

In [ ]:
def cvxopt_conelp(objf, lmis, variables):
    import cvxopt.solvers
    c, Gs, hs = lmi_sdp.to_cvxopt(objf, lmis, variables)
    cvxopt.solvers.options['feastol'] = epsilon_sdptol
    tic = time.time()
    sdpout = cvxopt.solvers.sdp(c, Gs=Gs, hs=hs)
    toc = time.time()
    print(sdpout['status'])
    print('Elapsed time: %.2f s'%(toc-tic))
    return numpy.matrix(sdpout['x'])

### [Raw SDPA file](http://plato.asu.edu/ftp/sdpa_format.txt)

Can be solved online with a *Semidefinite Programming* solver at [NEOS Solvers](http://www.neos-server.org/neos/solvers/index.html)

In [ ]:
def sdpa_file(objf, lmis, variables):
    sdpadat = lmi_sdp.to_sdpa_sparse(objf, lmis, variables)
    with open(tmpfolder + 'sdpa_dat/sdpa_input.dat-s', 'w') as f:
        f.write(sdpadat)
        
    print("SDPA file saved at: %ssdpa_dat/sdpa_input.dat-s"%tmpfolder)

--------------

Setting the default SDP solver:

In [ ]:
solve_sdp = cvxopt_dsdp5

## OLS Regression ($\hat{\beta}$)

In [ ]:
indentification_traj = 'traj1'

In [ ]:
regr_mats = numpy.load(tmpfolder + 'procdata/' + indentification_traj + '_regr.npz')
W = numpy.matrix(regr_mats['W'])
omega = numpy.matrix(regr_mats['omega'])
R1 = numpy.matrix(regr_mats['R1'])
rho1 = numpy.matrix(regr_mats['rho1'])

In [ ]:
omega.shape[0] / rbt.dof

In [ ]:
W.shape

In [ ]:
numpy.linalg.cond(W)

In [ ]:
%time beta_ols = (R1.T * R1).I * R1.T * rho1

In [ ]:
Path(tmpfolder + 'solutions').mkdir(parents=True, exist_ok=True)
numpy.savetxt(tmpfolder + 'solutions/' + indentification_traj + '_beta_ols.dat', beta_ols)

In [ ]:
rho2_norm_sqr = numpy.linalg.norm(omega - W * beta_ols)**2

### Base Parameter Feasibility Test (BPFT) of OLS Solution

In [ ]:
dict_subs = dict(zip(beta_symbs, numpy.asarray(beta_ols).flatten()))
lmis_ols_bpft = [ lmi_sdp.LMI_PD(mrepl(lmi.canonical().gts, dict_subs)) for lmi in DB_LMIs ]

In [ ]:
lmis_ols_bpft[0]

In [ ]:
variables_ols_bpft = list(delta_d)
objf_ols_bpft = 0  # no objective function - "find" problem

Testing with all solvers

**Just one of the following calls is sufficient to show feasibility/infeasibility**

In [ ]:
Path(tmpfolder + 'sdpa_dat').mkdir(parents=True, exist_ok=True)

In [ ]:
sol = sdpa_file(objf_ols_bpft, lmis_ols_bpft, variables_ols_bpft) # One can solve the output file at http://www.neos-server.org/neos/solvers/sdp:DSDP/SDPA.html for example.

In [ ]:
sol = dsdp5(objf_ols_bpft, lmis_ols_bpft, variables_ols_bpft)

In [ ]:
sol = sdpa(objf_ols_bpft, lmis_ols_bpft, variables_ols_bpft)

In [ ]:
sol = csdp(objf_ols_bpft, lmis_ols_bpft, variables_ols_bpft)

In [ ]:
sol = cvxopt_conelp(objf_ols_bpft, lmis_ols_bpft, variables_ols_bpft)

In [ ]:
sol = cvxopt_dsdp5(objf_ols_bpft, lmis_ols_bpft, variables_ols_bpft)

## Base Parameter Feasibility Correction (BPFC)     ($\beta^\prime$)

In [ ]:
u = sympy.Symbol('u')
U_beta = BlockMatrix([[Matrix([u]),            (beta_ols - beta_symbs).T],
                      [beta_ols - beta_symbs,                 I(n_beta)]])
U_beta = U_beta.as_explicit()

In [ ]:
lmis_ols_bpfc = [LMI(U_beta)] + DB_LMIs_marg

In [ ]:
lmis_ols_bpfc[1]

In [ ]:
variables_ols_bpfc = [u] + list(beta_symbs) + list(delta_d)

In [ ]:
objf_ols_bpfc = u

In [ ]:
sol_ols_bpfc = solve_sdp(objf_ols_bpfc, lmis_ols_bpfc, variables_ols_bpfc)

In [ ]:
u_prime = sol_ols_bpfc[0,0]
beta_prime = sol_ols_bpfc[1:1+n_beta]
delta_d_prime = sol_ols_bpfc[1+n_beta:]

In [ ]:
u_prime

### Solution Double Check

In [ ]:
dict_subs = dict(zip(beta_symbs, beta_prime.flatten().tolist()[0]))
dict_subs.update(dict(zip(delta_d, delta_d_prime.flatten().tolist()[0])))

test = 'ok'
for i, DBi in enumerate(DB_blocks):
        m = numpy.matrix(mrepl(DBi, dict_subs)).astype(float)
        for v in numpy.linalg.eigvals( m ):
            if v <= 0.0:
                print(i, v)
                test = 'fail'
print(test)

In [ ]:
Path(tmpfolder + 'solutions').mkdir(parents=True, exist_ok=True)
numpy.savetxt(tmpfolder + 'solutions/' + indentification_traj + '_beta_prime.dat', beta_prime)

## Feasible Base Parameter Estimation with Ordinary Least Squares (FBPE-OLS)     ($\beta^\star$)

In [ ]:
u = sympy.Symbol('u')
U_rho = BlockMatrix([[Matrix([u - rho2_norm_sqr]), (rho1 - R1*beta_symbs).T],
                     [rho1 - R1*beta_symbs,                       I(n_beta)]])
U_rho = U_rho.as_explicit()

In [ ]:
lmis_fbpe_ols = [LMI(U_rho)] + DB_LMIs_marg

In [ ]:
variables_fbpe_ols = [u] + list(beta_symbs) + list(delta_d)

In [ ]:
objf_fbpe_ols = u

In [ ]:
sol_fbpe_ols = solve_sdp(objf_fbpe_ols, lmis_fbpe_ols, variables_fbpe_ols)

In [ ]:
u_star = sol_fbpe_ols[0,0]
beta_star = numpy.matrix(sol_fbpe_ols[1:1+n_beta])
delta_d_star = numpy.matrix(sol_fbpe_ols[1+n_beta:])

In [ ]:
u_star

### Solution Double Check

In [ ]:
dict_subs = dict(zip(beta_symbs, beta_star.flatten().tolist()[0]))
dict_subs.update(dict(zip(delta_d, delta_d_star.flatten().tolist()[0])))

test = 'ok'
for DBi in DB_blocks:
        m = numpy.matrix(mrepl(DBi, dict_subs)).astype(float)
        for v in numpy.linalg.eigvals( m ):
            if v <= 0.0:
                print(v)
                test = 'fail'
print(test)

In [ ]:
Path(tmpfolder + 'solutions').mkdir(parents=True, exist_ok=True)
numpy.savetxt(tmpfolder + 'solutions/' + indentification_traj + '_beta_star.dat', beta_star)

## Additional Constraints on Centers-of-Mass    ($\beta^{\star e}$)

In [ ]:
link_cuboid_hulls = [
[[ -0.140 , +0.140 ],[ -0.174 , +0.174 ],[ -0.084 , +0.346 ]],
[[ -0.084 , +0.084 ],[ -0.174 , +0.174 ],[ -0.084 , +0.170 ]],
[[ -0.090 , +0.040 ],[ -0.550 , +0.040 ],[ -0.045 , +0.045 ]],
[[ -0.045 , +0.095 ],[ -0.045 , +0.045 ],[ -0.050 , +0.830 ]],
[[ -0.045 , +0.045 ],[ -0.020 , +0.100 ],[ -0.045 , +0.045 ]],
[[ -0.045 , +0.045 ],[ -0.060 , +0.045 ],[ -0.020 , +0.060 ]],
[[ -0.045 , +0.045 ],[ -0.045 , +0.045 ],[ -0.018 , +0.001 ]],
]

robotmaxmass = 27.0

#### LMI Matrix Definition

In [ ]:
D_add_blocks = []
for i in range(rbt.dof):
    l = rbt.rbtdef.l[i]
    m = rbt.rbtdef.m[i]
    link_cuboid_hull = link_cuboid_hulls[i]
    for j in range(3):
        D_add_blocks.append( Matrix( [[  l[j] - m*link_cuboid_hull[j][0] ]] ) )
        D_add_blocks.append( Matrix( [[ -l[j] + m*link_cuboid_hull[j][1] ]] ) )

D_add_blocks.append(Matrix([robotmaxmass - sum(rbt.rbtdef.m)]))

#### Variable Change and LMI Definition

In [ ]:
DB_add_blocks = [ mrepl(Di, varchange_dict) for Di in D_add_blocks ]

In [ ]:
DB_add_LMIs = list(map(LMI_PD, DB_add_blocks))

In [ ]:
DB_add_LMIs_marg = list(map(lambda lm: LMI(lm, epsilon_safemargin*eye(lm.shape[0])) , DB_add_blocks))

In [ ]:
DB_add_LMIs_marg[0]

In [ ]:
DB_add_LMIs_marg[-1]

### Testing $\beta^\star$ solution on new constraints

In [ ]:
dict_subs = dict(zip(beta_symbs, numpy.asarray(beta_star).flatten()))
lmis_star_bpft_add = [ lmi_sdp.LMI_PD(mrepl(lmi.canonical().gts, dict_subs)) for lmi in DB_add_LMIs_marg ]
variables_star_bpft_add = list(delta_d)
objf_star_bpft_add = delta_d[0]

In [ ]:
sol_star_bpft_add = solve_sdp(objf_star_bpft_add, lmis_star_bpft_add, variables_star_bpft_add)

### Finding new solution

In [ ]:
lmis_fbpe_ols_add = [LMI(U_rho)] + DB_LMIs_marg + DB_add_LMIs_marg

In [ ]:
variables_fbpe_ols_add = [u] + list(beta_symbs) + list(delta_d)

In [ ]:
objf_fbpe_ols_add = u

In [ ]:
sol_fbpe_ols_add = solve_sdp(objf_fbpe_ols_add, lmis_fbpe_ols_add, variables_fbpe_ols_add)

In [ ]:
u_starextra = sol_fbpe_ols_add[0,0]
beta_starextra = numpy.matrix(sol_fbpe_ols_add[1:1+n_beta])
delta_d_starextra = numpy.matrix(sol_fbpe_ols_add[1+n_beta:])

In [ ]:
u_starextra

### Solution Double Check

In [ ]:
dict_subs = dict(zip(beta_symbs, beta_starextra.flatten().tolist()[0]))
dict_subs.update(dict(zip(delta_d, delta_d_starextra.flatten().tolist()[0])))

test = 'ok'
for DBi in (DB_blocks + DB_add_blocks):
        m = numpy.matrix(mrepl(DBi, dict_subs)).astype(float)
        for v in numpy.linalg.eigvals( m ):
            if v <= 0.0: test = 'fail'
print(test)

In [ ]:
Path(tmpfolder + 'solutions').mkdir(parents=True, exist_ok=True)
numpy.savetxt(tmpfolder + 'solutions/' + indentification_traj + '_beta_starextra.dat', beta_starextra)

### Inertia Matrix Definiteness Tests

In [ ]:
def test_mass_matrix_psd(beta_solution):
    K = rbt.dyn.Pb.T + rbt.dyn.Kd * rbt.dyn.Pd.T
    A = numpy.matrix(K).astype(float)
    b = numpy.matrix(beta_solution).astype(float)
    x = numpy.linalg.pinv(A) * b
    d_test = x
    
    M_func_def = sympybotics.robotcodegen.robot_code_to_func( 'python', rbt.M_code, 'M', 'M_func', rbt.rbtdef)
    exec(M_func_def, globals())
    global sin, cos
    from math import sin, cos
    
    ntests = 10000
    
    ok = 0
    nok = 0
    for i in range(ntests):
        q_test = numpy.random.rand(rbt.dof)
        M_out = M_func( numpy.array(d_test).flatten().tolist(), q_test)
        M = numpy.matrix(M_out).reshape((rbt.dof,rbt.dof))
        try:
            c = numpy.linalg.cholesky(M)
        except:
            nok += 1
            continue
        ok += 1
        
    print('ok     %8i  %3i%%'%(ok, 100.0*ok/ntests))
    print('not ok %8i  %3i%%'%(nok, 100.0*nok/ntests))
    if nok > 0 :
        print('\nNot PSD -> Not feasible!')
    else:
        print('\nSeems feasible.')

In [ ]:
beta_ols = numpy.matrix(numpy.loadtxt(tmpfolder + 'solutions/traj1_beta_ols.dat')).T
test_mass_matrix_psd(beta_ols)

In [ ]:
beta_starextra = numpy.matrix(numpy.loadtxt(tmpfolder + 'solutions/traj1_beta_starextra.dat')).T
test_mass_matrix_psd(beta_starextra)

5 - Regression Model Validation
====================================================

In [ ]:
import numpy as np
import sympybotics
import pickle

from collections import OrderedDict
from support_funcs.utils import ListTable

In [ ]:
with open(tmpfolder +  'robotmodels/wam7_model.pkl', 'rb' ) as file:
          rbt = pickle.load( file )

In [ ]:
def analyse(W, omega, R1, beta):
    from numpy import matrix, mean
    from numpy.linalg import cond, norm
    
    p = dict()
    
    n = W.shape[0]
    
    omega_norm = norm(omega)
    omega_mean = mean(omega)
    
    p['err'] = norm(omega - W * beta)
    p['merr'] = p['err'] / n
    
    p['se'] = p['err']**2
    p['mse'] = p['se']/(n-W.shape[1])
    p['rmse'] = p['mse']**0.5
    
    C = p['mse'] * (R1.T * R1).I
    p['sd'] = np.sqrt(C.diagonal()).T
    p['sd%'] = 100. * p['sd'] / np.abs(beta)
    
    p['relerr'] = p['err']/ omega_norm
    p['relerr%'] = p['relerr']*100.
    
    p['1-r2'] = p['err']**2 / norm(omega - omega_mean)**2
    p['r2'] = 1 - p['1-r2']

    return p
    

In [ ]:
trajs = OrderedDict([('id', 'traj1'),
                     ('vA', 'traj2'),
                     ('vB', 'traj3'),
                     ('vC', 'traj4')])


betas = OrderedDict([('beta_ols', 'B^'),
                     ('beta_prime', 'B\''),
                     ('beta_star', 'B*'),
                     ('beta_starextra', 'B*e')])

In [ ]:
solutions_beta = {betaname:np.matrix(np.loadtxt(tmpfolder + 'solutions/%s_%s.dat'%(trajs['id'], betaname))).T for betaname in betas}

In [ ]:
analysis = dict()

for traj in trajs:
    from numpy.linalg import cond
    
    print(traj)
    
    analysis[traj] = dict()
    
    regr_mats = np.load(tmpfolder + 'procdata/' + trajs[traj] + '_regr.npz')
    W = np.matrix(regr_mats['W'])
    omega = np.matrix(regr_mats['omega'])
    R1 = np.matrix(regr_mats['R1'])
    
    analysis[traj]['cond'] = cond(W)
    
    analysis[traj]['betas'] = dict()
    
    for betaname in betas:
        beta = solutions_beta[betaname]
        analysis[traj]['betas'][betaname] = analyse(W, omega, R1, beta)
    
    del W, omega, R1

In [ ]:
prop = lambda x: x['relerr%']
form = '%.2f'

table = ListTable()
table.append(['','cond']+list(betas.values()))
for (traj, trajname) in zip(trajs, ['identification', 'validation A', 'validation B', 'validation C']):
    row = [traj] + ['%.0f'%analysis[traj]['cond']] + [ form%prop(analysis[traj]['betas'][b]) for b in betas]
    table.append(row)
table

In [ ]:
import yaml, sympy
with open(datafolder + 'robotparams/wam7_cad.yml', 'r') as f:
    wam7_cad = yaml.safe_load(f)
delta_cad = sympy.Matrix([wam7_cad.get(str(d), d) for d in rbt.dyn.dynparms])
beta_cad = (rbt.dyn.Pb.T + rbt.dyn.Kd * rbt.dyn.Pd.T) * delta_cad

In [ ]:
form = '%.4g'

import sympy

table = ListTable()
header = ['', 'Bcad', 'B^', '% std dev B^', 'B*e']
table.append(header)
for i, b in enumerate(rbt.dyn.baseparms):
    if beta_cad[i].is_Number:
        cad_v = sympy.N(beta_cad[i], 4)
    elif beta_cad[i].is_Symbol:
        cad_v = '---'
    else:
        cad_v = sympy.N(beta_cad[i], 4)
    row = ['%.7s ...'%b if len(str(b)) > 7 else str(b), cad_v]
    row += [form%solutions_beta['beta_ols'][i,0], '%.2g'%analysis['id']['betas']['beta_ols']['sd%'][i,0]]
    row += [form%solutions_beta['beta_starextra'][i,0]]
    table.append(row)
table

## Torque Plot

In [ ]:
traj = 'vA'
betaname = 'beta_starextra'

In [ ]:
regr_mats = np.load(tmpfolder + 'procdata/' + trajs[traj] + '_regr.npz')
W = np.matrix(regr_mats['W'])
omega = np.matrix(regr_mats['omega'])

regr_mats = np.load(tmpfolder + 'procdata/' + trajs[traj] + '_proc.npz')
t = regr_mats['t']
tau_proc = np.matrix(regr_mats['tau'])

regr_mats = np.load(tmpfolder + 'procdata/' + trajs[traj] + '_raw.npz')
tau_raw = np.matrix(regr_mats['tau'])

In [ ]:
beta_solution = np.matrix(np.loadtxt(tmpfolder + 'solutions/%s_%s.dat'%(trajs['id'], betaname))).T

In [ ]:
tau = tau_raw

In [ ]:
omega_pred = W * beta_solution
tau_pred = omega_pred.reshape( round(omega_pred.shape[0]/tau.shape[1]), tau.shape[1] )
err_pred = tau - tau_pred

In [ ]:
joint = 7
firsts = 20000
s=tau_raw.shape[0]

In [ ]:
from matplotlib import pyplot as plt
plt.close()

plt.figure(figsize=(12,8))

axes = []
for i in range(tau.shape[1]):
    ax = plt.subplot(3, 3, i+1)
    ax.plot(t[:firsts],tau_raw[:,i][:firsts], label="Measured torque")
    ax.plot(t[:firsts],tau_pred[:,i][:firsts], label="Estimated torque")
    ax.plot(t[:firsts],err_pred[:,i][:firsts], label="Error")
    ax.set_title( "Joint %d"%(i+1) )
    plt.xlabel("Time (s)")
    plt.ylabel("Torque (Nm)")
    axes.append(ax)


plt.tight_layout()
plt.legend(loc='upper left', bbox_to_anchor=(1.2, 1))

plt.show()

---------------------------

---------------------------

---------------------------

---------------------------

---------------------------

---------------------------

---------------------------

---------------------------

---------------------------

### (generate readme and notebook preview)

In [ ]:
def save_files(): 
    from IPython.display import display, Javascript
    Javascript('IPython.notebook.save_notebook()')
    
    import json
    with open('WAM7 Dynamic Parameter Identification.ipynb', 'r') as f:
        d = json.load(f)
    with open('README.md', 'w') as f:
        f.write(''.join(d['worksheets'][0]['cells'][0]['source']).encode('UTF-8'))
    
    !ipython nbconvert "WAM7 Dynamic Parameter Identification.ipynb" --to html

In [ ]:
#save_files()

In [ ]:
#!git status

In [ ]:
#!git commit -a -m "Add automathic README.md file generator"

In [ ]:
#!git push

## Equivalent full dynamics parameters

In [ ]:
Pb = numpy.matrix(rbt.dyn.Pb).astype(float)
Pd = numpy.matrix(rbt.dyn.Pd).astype(float)
Kd = numpy.matrix(rbt.dyn.Kd).astype(float)

beta_delta_d = numpy.concatenate((beta_starextra, delta_d_starextra), axis=0)

# second line of Eq (49) needs to be K_G^-1 (instead of G_K^-1) and the experession for that with -K_d does not hold (numerically tested)
P = numpy.concatenate((Pb, Pd), axis=1)
K_G = numpy.block([[numpy.eye(n_beta), Kd],
                   [numpy.zeros((n_delta_d, n_beta)), numpy.eye(n_delta_d)]])

delta = (K_G*P.T).I*beta_delta_d

In [ ]:
delta = delta.flatten().tolist()[0]
dict(zip(rbt.rbtdef.dynparms(), delta))

# TMECH Extension
C. D. Sousa and R. Cortesão, "Inertia Tensor Properties in Robot Dynamics Identification: A Linear Matrix Inequality Approach," in IEEE/ASME Transactions on Mechatronics, vol. 24, no. 1, pp. 406-411, Feb. 2019, doi: [10.1109/TMECH.2019.2891177](http://dx.doi.org/10.1109/TMECH.2019.2891177).

#### LMI Matrix Definition for Triangle Inequalities

In [ ]:
from sympy import Trace

D_triangle_blocks = []
for i in range(rbt.dof):
    l = rbt.rbtdef.l[i]
    m = rbt.rbtdef.m[i]
    L = rbt.rbtdef.L[i]
    Di = BlockMatrix([[Trace(L)*I(3)/2 - L,      l],
                      [l.T,                 I(1)*m]])
    D_triangle_blocks.append(Di.simplify().as_explicit())

In [ ]:
D_triangle_blocks[0]

#### Variable Change and LMI Definition

In [ ]:
DB_triangle_blocks = [ mrepl(Di, varchange_dict) for Di in D_triangle_blocks ]

In [ ]:
DB_triangle_LMIs = list(map(LMI_PD, DB_triangle_blocks))

In [ ]:
DB_triangle_LMIs_marg = list(map(lambda lm: LMI(lm, epsilon_safemargin*eye(lm.shape[0])) , DB_triangle_blocks))

In [ ]:
DB_triangle_LMIs_marg[0]

### Finding new solution

In [ ]:
lmis_fbpe_ols_triangle = [LMI(U_rho)] + DB_LMIs_marg + DB_add_LMIs_marg + DB_triangle_LMIs_marg

In [ ]:
variables_fbpe_ols_triangle = [u] + list(beta_symbs) + list(delta_d)

In [ ]:
objf_fbpe_ols_triangle = u

In [ ]:
sol_fbpe_ols_triangle= solve_sdp(objf_fbpe_ols_triangle, lmis_fbpe_ols_triangle, variables_fbpe_ols_triangle)

In [ ]:
u_triangle = sol_fbpe_ols_triangle[0,0]
beta_triangle = numpy.matrix(sol_fbpe_ols_triangle[1:1+n_beta])
delta_d_triangle = numpy.matrix(sol_fbpe_ols_triangle[1+n_beta:])

In [ ]:
u_triangle

In [ ]:
form = '%.4g'

import sympy

table = ListTable()
header = ['', 'Bcad', 'B^', '% std dev B^', 'B*e', 'B△e']
table.append(header)
for i, b in enumerate(rbt.dyn.baseparms):
    if beta_cad[i].is_Number:
        cad_v = sympy.N(beta_cad[i], 4)
    elif beta_cad[i].is_Symbol:
        cad_v = '---'
    else:
        cad_v = sympy.N(beta_cad[i], 4)
    row = ['%.7s ...'%b if len(str(b)) > 7 else str(b), cad_v]
    row += [form%solutions_beta['beta_ols'][i,0], '%.2g'%analysis['id']['betas']['beta_ols']['sd%'][i,0]]
    row += [form%solutions_beta['beta_starextra'][i,0]]
    row += [form%beta_triangle[i,0]]
    table.append(row)
table